# Python for Learning AI Week 2: APIs and Web Communication

Welcome to Week 2 of our Python development series! This week, we'll dive into working with APIs (Application Programming Interfaces) in Python. By the end of this session, you'll understand how to interact with web services, handle API responses, and build robust API integrations.

**Restaurant Metaphor**: Throughout this notebook, we'll compare APIs to a restaurant experience:
- The API is like a restaurant kitchen (the backend)
- You (the client) are a customer placing orders
- Your API requests are like ordering from a menu
- The waiter is the intermediary handling your requests
- API responses are like the food being delivered to your table

This metaphor will help make abstract API concepts more concrete and relatable!

## Table of Contents
- [What You'll Learn](#what-youll-learn)
- [Prerequisites](#prerequisites)
- [Package Management](#package-management)
- [Understanding APIs as Restaurant Services](#understanding-apis)
  - [The Role of JSON in APIs](#json-role)
- [Making HTTP Requests (Placing Orders)](#making-http-requests)
- [Handling Authentication (Restaurant Membership)](#handling-authentication)
- [Error Handling and Best Practices (Restaurant Problem Management)](#error-handling-and-best-practices)
- [Practice Exercise (Build Your Own Restaurant)](#practice-exercise)

## What You'll Learn
1. Understanding APIs and their role in modern software
2. Making HTTP requests with Python's `requests` library
3. Working with REST APIs and JSON data
4. Handling API authentication and errors
5. Building a simple API client

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of dictionaries and JSON
- Familiarity with functions and error handling

Let's start by installing the required packages:

## A Note on Package Management

**Restaurant equivalent**: Before you can place orders at a restaurant, you need to make sure the restaurant has all the ingredients for the menu items (packages) you want to order.

### Why Package Management is Essential

Package management is absolutely critical for successful API development and production code:

1. **Reproducibility**: Ensures your code works the same way across different environments
2. **Dependency Resolution**: Automatically handles complex package relationships
3. **Version Control**: Prevents conflicts between packages with incompatible versions
4. **Collaboration**: Makes it easy for team members to run the same code
5. **Security**: Helps maintain updated packages with security patches

For this course, we use `uv` - a modern, faster Python package installer and resolver.

### Key Benefits of `uv`

- **Speed**: Significantly faster than traditional pip (often 10-100x faster)
- **Reliability**: Improved dependency resolution with fewer conflicts
- **Reproducibility**: Precise version locking for consistent environments
- **Compatibility**: Works with existing requirements files and pyproject.toml

### First: Sync Your Environment

**BEFORE YOU BEGIN**: Always sync your virtual environment with the latest project dependencies:

```bash
# Run this in your terminal before starting work
uv pip sync pyproject.toml
```

This command:
- Updates all packages to match versions in the project configuration (ensures the kitchen has the right ingredients)
- Installs any new dependencies that were added since you last worked (stocks up on new ingredients)
- Removes packages no longer needed by the project (clears out expired ingredients)
- Ensures everyone has the same development environment (all chefs work with the same ingredients)

### Package Management Best Practices

1. **DO NOT** install packages using `!pip install` in notebooks
   - This only installs to the current kernel (like buying ingredients just for one meal)
   - Changes are not persistent across project (ingredients disappear after dinner)
   - Can lead to inconsistencies (different chefs using different ingredients)

2. **DO** install packages using `uv` in your terminal:
   ```bash
   uv pip install package_name
   ```
   - This properly updates your project's inventory (adds to the restaurant's permanent stock)
   - Ensures consistency across the project (all chefs use the same ingredients)

### Need More Help?

- For more details on Python package management, refer to [Week 1 materials](../week1/week1_python_basics.ipynb)
- Ask ChatGPT: "How does uv package manager work in Python?" or "Explain Python virtual environments"
- Check out [uv documentation](https://github.com/astral-sh/uv) for advanced features

In [ ]:
# Import libraries we'll use
import requests
import json

## 1. Understanding APIs

An API (Application Programming Interface) is like a contract between different software components. Think of it as a waiter in a restaurant:
- You (the client) don't need to know how the kitchen (the server) works
- You just need to know how to place an order (make a request)
- The waiter (API) handles communication between you and the kitchen

### Common API Terms:
- **Endpoint**: The URL where an API can be accessed
- **Request**: What you send to the API
- **Response**: What the API sends back
- **HTTP Methods**: GET, POST, PUT, DELETE, etc.
- **Status Codes**: 200 (OK), 404 (Not Found), etc.

### The Role of JSON in APIs <a id="json-role"></a>

JSON (JavaScript Object Notation) is the primary language that modern APIs use to communicate. It's like the common language that both you and the restaurant staff understand:

**Restaurant equivalent**: JSON is like the standard menu format that makes it easy for both customers and kitchen staff to understand orders.

Key aspects of JSON in APIs:
- **Data Format**: JSON provides a standardized way to structure data (similar to Python dictionaries and lists)
- **Human-Readable**: Unlike binary formats, JSON is easy for humans to read and debug
- **Language Independent**: Works across programming languages (Python, JavaScript, Java, etc.)
- **Lightweight**: Adds minimal overhead to API requests and responses
- **Flexible**: Can represent complex nested data structures

A typical JSON response from an API looks like:
```json
{
  "id": 1,
  "title": "Our Special Dish",
  "description": "A delicious specialty",
  "price": 12.99,
  "available": true,
  "ingredients": ["tomato", "basil", "mozzarella"]
}
```

In Python, we use the `json` module to:
- **Parse JSON responses**: Convert JSON strings to Python dictionaries/lists with `json.loads()`
- **Create JSON requests**: Convert Python objects to JSON strings with `json.dumps()`

Let's start with a simple example using a public API:

In [ ]:
# Let's try a simple API call to JSONPlaceholder (a free testing API)
response = requests.get('https://jsonplaceholder.typicode.com/posts/1')

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response - this converts the JSON string to a Python dictionary
    data = response.json()
    print("API Response (Python dictionary):")
    print(data)
    
    print("\nJSON Response (formatted):")
    # json.dumps() converts a Python object back to a JSON string with formatting
    print(json.dumps(data, indent=2))
    
    # Accessing specific values from the JSON response
    print("\nAccessing specific fields:")
    print(f"Title: {data['title']}")
    print(f"User ID: {data['userId']}")
else:
    print(f"Error: Status code {response.status_code}")

# Let's look at the response headers
print("\nResponse Headers:")
for key, value in response.headers.items():
    print(f"{key}: {value}")

# Let's check the Content-Type header specifically
print(f"\nContent-Type: {response.headers.get('Content-Type')}")

## 2. Making Different Types of HTTP Requests

APIs use different HTTP methods for different operations, similar to how a waiter handles different types of requests in a restaurant:

- **GET**: Retrieve data (Like asking the waiter "What's today's special?")
- **POST**: Create new data (Like ordering a new dish: "I'd like the pasta, please.")
- **PUT/PATCH**: Update existing data (Like modifying an order: "Could I get that pasta with chicken instead?") 
- **DELETE**: Remove data (Like canceling an order: "I'd like to cancel the dessert, please.")

### JSON in API Requests and Responses

For each HTTP method, JSON plays a specific role:

- **GET requests**: JSON is typically only in the response (the data you receive)
- **POST/PUT requests**: JSON is used in both the request body (the data you send) and the response
- **All responses**: Most modern APIs return data in JSON format, with appropriate HTTP status codes

Let's set up a base URL for our examples - think of this as choosing which restaurant we'll be visiting:

In [ ]:
# Define our base URL for subsequent API calls - this is like choosing which restaurant to visit
base_url = 'https://jsonplaceholder.typicode.com/posts'
print(f"Restaurant API (Kitchen access point): {base_url}")

# Let's see what a JSON structure looks like manually - like a standard order form
example_menu_item = {
    "title": "Margherita Pizza",
    "body": "Classic pizza with tomato, mozzarella, and basil",
    "userId": 1  # The chef who prepares it
}

# Convert Python dict to JSON string for demonstration - like translating an order to kitchen notation
json_string = json.dumps(example_menu_item, indent=2)
print("\nJSON representation of a menu item (the kitchen's language):")
print(json_string)

# We could convert it back to a Python dict like this - like translating from kitchen notation back to waiter's language
python_dict = json.loads(json_string)
print("\nBack to Python dictionary (the waiter's language):")
print(python_dict)

### GET Request: Retrieving Data

The GET method is used to retrieve data from a server. Continuing our restaurant metaphor:

**Restaurant equivalent**: Asking your waiter "Can you tell me about the soup of the day?"

The waiter (API) goes to the kitchen (server), gets information, and returns with details about the soup without changing anything in the kitchen.

Let's create a function to get a post by its ID:

In [ ]:
# GET request (read data) - like asking about a menu item
def get_menu_item(item_id):
    """Retrieve a menu item by its ID using GET request"""
    response = requests.get(f'{base_url}/{item_id}')
    return response.json() if response.ok else None

# Let's try it - asking the waiter about menu item #1
menu_item = get_menu_item(item_id=1)
print("GET Request Result (Menu Item Information):")
print(json.dumps(menu_item, indent=2))

### POST Request: Creating Data

The POST method is used to send data to create a new resource. 

**Restaurant equivalent**: Placing a new order with the waiter.

You provide all the details (what dish you want, how you want it cooked, any special requests), and the waiter takes this information to the kitchen where a new dish is created specifically for you. The kitchen (server) responds by creating something new and sending back confirmation (like an order number or estimated time).

**JSON in POST Requests**:
- Your data needs to be sent in a format the server understands (JSON)
- The `requests` library makes this easy with the `json` parameter
- The server processes the JSON data and usually responds with JSON

Let's create a function to create a new post:

In [ ]:
# POST request (create data) - like placing a new order
def place_order(dish_name, special_instructions, chef_id):
    """Place a new order with the kitchen"""
    # Create a Python dictionary with our order details
    new_order = {
        'title': dish_name,
        'body': special_instructions,
        'userId': chef_id  # The chef who will prepare it
    }
    
    # The 'json' parameter automatically:
    # 1. Converts the Python dict to a JSON string (translates to kitchen language)
    # 2. Sets the Content-Type header to 'application/json' (specifies the order format)
    # 3. Sends the JSON in the request body (sends the order slip to the kitchen)
    response = requests.post(base_url, json=new_order)
    
    return response.json() if response.ok else None

# Let's try it - placing an order for spaghetti carbonara
new_order = place_order(
    dish_name='Spaghetti Carbonara',
    special_instructions='Extra cheese please, light on pepper',
    chef_id=1  # Chef Mario is preparing this dish
)
print("POST Request Result (Order Confirmation):")
print(json.dumps(new_order, indent=2))

# Let's look at what we sent vs. what we got back
print("\nWhat's happening behind the scenes (Restaurant Process):")
print("1. We created an order in the waiter's language (Python dictionary)")
print("2. The waiter translated it to kitchen language (JSON string)")
print("3. The kitchen received the order slip and started preparing (API created resource)")
print("4. The kitchen sent back an order confirmation (JSON response)")
print("5. The waiter translated the confirmation for us (back to Python dictionary)")

### PUT Request: Updating Data

The PUT method is used to update an existing resource.

**Restaurant equivalent**: Modifying an order that's already been placed.

Imagine you've ordered a steak (medium rare), but then change your mind. You call the waiter back and say, "I'd like to change my steak to well done and add a side of vegetables." You're updating an existing order, not creating a new one. The waiter takes your complete revised order to the kitchen, replacing the old instructions entirely.

Let's create a function to update a post:

In [ ]:
# PUT request (update data) - like modifying an existing order
def modify_order(order_id, new_dish_name, new_instructions, chef_id):
    """Modify an existing order that's already been placed"""
    updated_order = {
        'id': order_id,
        'title': new_dish_name,
        'body': new_instructions,
        'userId': chef_id
    }
    response = requests.put(f'{base_url}/{order_id}', json=updated_order)
    return response.json() if response.ok else None

# Let's try it - changing our pasta order to a different dish
modified_order = modify_order(
    order_id=1,  # The original order number
    new_dish_name='Fettuccine Alfredo',  # What we want instead
    new_instructions='With chicken and broccoli please',  # How to prepare it
    chef_id=1  # Still prepared by Chef Mario
)
print("PUT Request Result (Modified Order):")
print(json.dumps(modified_order, indent=2))

### DELETE Request: Removing Data

The DELETE method is used to remove a resource.

**Restaurant equivalent**: Canceling an order completely.

You realize you need to leave soon, so you tell the waiter, "I need to cancel my dessert order." The waiter goes to the kitchen and removes that order from the system entirely. The kitchen acknowledges the cancellation and stops any preparation that might have started.

Let's create a function to delete a post:

In [ ]:
# DELETE request - like canceling an order
def cancel_order(order_id):
    """Cancel an order completely by its ID"""
    response = requests.delete(f'{base_url}/{order_id}')
    return response.ok

# Let's try it - canceling our order #1
success = cancel_order(order_id=1)  # Cancel by order number
print(f"DELETE Request Result (Order Cancellation): Success = {success}")


## After Deletion

When a resource is deleted through a DELETE request (like canceling an order at a restaurant), that resource no longer exists on the server. Any subsequent attempts to retrieve (GET) or modify (PUT) that resource will typically fail with a 404 Not Found error or similar, as the server can no longer locate the requested resource. This is an important aspect of RESTful API design - operations have real consequences that affect subsequent requests.


In [ ]:

# Now let's try to retrieve the deleted order
try:
    print("\nAttempting to retrieve the canceled order:")
    deleted_item = get_menu_item(item_id=1)  # Try to get the canceled order
    print(f"Retrieved item (should fail): {deleted_item}")
except Exception as e:
    print(f"Error retrieving deleted order: {e}")

# Let's also try modifying the deleted order
try:
    print("\nAttempting to modify the canceled order:")
    modified_order = modify_order(
        order_id=1,  # Trying to modify the canceled order
        new_dish_name="Something else",
        new_instructions="This won't work",
        chef_id=2
    )
    print(f"Modified item (should fail): {modified_order}")
except Exception as e:
    print(f"Error modifying deleted order: {e}")


## 3. Handling Authentication and User Data

Many APIs require authentication and track user information. In our restaurant metaphor, this is not just about verifying who you are, but also tracking your orders, preferences, and providing personalized service.

### Authentication Methods

**Restaurant equivalents**:
- **API Keys**: Like having a membership card to an exclusive restaurant (The waiter checks your card before taking your order)
- **OAuth tokens**: Similar to making a reservation under someone else's name who has special privileges (The waiter verifies with the manager that you're allowed to use these privileges)
- **Basic authentication**: Like providing your name and phone number to confirm your reservation (Simple identity verification)

> **Note**: These authentication methods are essential when working with Large Language Model (LLM) APIs like OpenAI, Anthropic, Google Gemini, and other AI services. Most LLM providers require API keys for authentication, usage tracking, and billing purposes. We'll explore the specifics of LLM API authentication in a dedicated notebook next week.

### Why Authentication and User Tracking Matter:

- **Security**: Protects the kitchen's secret recipes (sensitive data)
- **Access Control**: Only paying customers get food (authorized users access resources)
- **Personalization**: The restaurant remembers your preferences (customized user experience)
- **Order Attribution**: Knowing who ordered what (associating data with specific users)
- **Loyalty Programs**: Regular customers get benefits (premium API features for subscribers)
- **Data Privacy**: Protecting customer information (secure handling of user data)


### User Data in APIs: Benefits and Responsibilities

**Benefits of User Identification**:
- **Order History**: The restaurant tracks your past orders for quicker reordering
- **Preference Learning**: "The usual, sir?" - Systems learn your preferences over time
- **Loyalty Rewards**: Frequent customers get special discounts or faster service
- **Personalized Recommendations**: "Based on your taste profile, you might enjoy our new dish"

**Data Privacy Responsibilities**:
- **Data Minimization**: Only collect what you need ("We only ask for your name, not your life story")
- **Secure Storage**: Encrypt sensitive information ("Your credit card is stored in our secure safe")
- **Clear Policies**: Transparent data usage ("Here's how we use your information")
- **User Control**: Allow data access and deletion ("You can review or remove your information anytime")
- **Compliance**: Follow regulations like GDPR, CCPA ("We follow restaurant industry standards for customer privacy")

Let's see examples of basic authentication:

In [ ]:
# Example with API Key 
def api_key_example():
    """Using an API key in the Authorization header"""
    api_key = "your_api_key_here"  # In real code, get this from environment variables
    
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    
    # This is just an example URL
    print("Using API key authentication:")
    print(f"  Authorization: Bearer {api_key[:3]}...{api_key[-3:]}" if len(api_key) > 10 else f"  Authorization: Bearer {api_key}")
    
    # In actual use: response = requests.get('https://api.example.com/data', headers=headers)

# Example with Basic Auth
def basic_auth_example():
    """Using username/password authentication"""
    username = "restaurant_user"
    password = "secure_password"  # In real code, get these from environment variables
    
    print("Using Basic authentication:")
    print(f"  With requests: requests.get(url, auth=('{username}', '{password[:2]}****'))")
    
    # In actual use: response = requests.get('https://api.example.com/data', auth=(username, password))

# Run the examples
api_key_example()
print()
basic_auth_example()

## Environment Variables: Best Practices for API Development

When working with APIs, you'll often need to manage sensitive information like:
- API keys
- Authentication tokens
- Credentials
- Database connection strings
- Server configurations

**Restaurant equivalent**: Think of these as your personal information at a restaurant:
- Your membership card or loyalty number (API keys)
- Your reservation confirmation (authentication tokens)
- Your credit card details (credentials)
- Your preferred table location (connection settings)
- Your dietary restrictions (configuration)

You wouldn't want to shout this information across the restaurant (hardcoding in your application), nor would you want it written on a public whiteboard (committed to a public repository).

### Using Environment Variables in Python

Environment variables are key-value pairs that exist outside your application code. They're an ideal solution for managing configuration:

**Restaurant equivalent**: This is like keeping your personal information in your wallet instead of written on your shirt. You can still access it when needed, but it's not visible to everyone.

1. **Benefits:**
   - Separates code from configuration (your meal preference isn't printed on the menu)
   - Improves security by keeping sensitive data out of code (credit card stays in your wallet)
   - Allows different settings across environments (different preferences at different restaurants)

2. **Common Python libraries:**
   - `os` (built-in): Basic environment variable access
   - `dotenv`: Loads variables from .env files
   - `python-decouple`: More advanced configuration management

Let's see how to use them:

In [ ]:
# Basic environment variable handling with os module
import os

# Example: Reading environment variables
def get_api_key():
    """Get API key from environment variable or use a default for development"""
    api_key = os.environ.get('API_KEY')
    if api_key is None:
        print("Warning: API_KEY not found in environment variables!")
        print("Using a placeholder value - this won't work for real APIs")
        return "placeholder_api_key_for_development_only"
    return api_key

# Example of how you would use this in practice
api_key = get_api_key()
print(f"API Key: {api_key}")

# Check for other common environment variables
print("\nCommon environment variables available:")
for env_var in ['USER', 'HOME', 'PATH', 'PYTHON_VERSION']:
    value = os.environ.get(env_var)
    if value:
        # Only show first 30 characters for readability
        display_value = value[:30] + '...' if len(value) > 30 else value
        print(f"- {env_var}: {display_value}")
    else:
        print(f"- {env_var}: Not set")

### Using .env Files

For development, it's common to use `.env` files to store environment variables. The `python-dotenv` package makes this easy.

**Restaurant equivalent**: This is like having your preferences stored on your profile in the restaurant's system. When you arrive, they can look up your preferences without you having to state them each time.

#### Environment Variables Best Practice

We've created a `.env.example` file in the project root that contains all the environment variables used in this project:

```
# API Authentication and Configuration
# --------------------------------
# Your API Key for service authentication (never commit the real key)
API_KEY=your_secret_key_here

# The base URL for your API service
API_URL=https://api.example.com/v1

# Optional API Configuration
# -------------------------
# Timeout for API requests in seconds
API_TIMEOUT=30

# Debug mode (true/false)
DEBUG=False

# Additional service-specific variables
...
```

**To get started with environment variables:**

1. Copy the `.env.example` file to create your own `.env` file:
   ```bash
   cp .env.example .env
   ```
2. Edit your new `.env` file with your actual values
3. Never commit your `.env` file to version control (it should be in your `.gitignore`)

This approach provides documentation for all required environment variables while keeping sensitive values secure.

In [ ]:
# Import and setup python-dotenv
try:
    from dotenv import load_dotenv
    import os
    
    # Check if .env file exists, if not, suggest creating from .env.example
    env_path = os.path.join(os.path.dirname(os.getcwd()), '.env')
    env_example_path = os.path.join(os.path.dirname(os.getcwd()), '.env.example')
    
    if not os.path.exists(env_path) and os.path.exists(env_example_path):
        print("⚠️ No .env file found! We've provided a .env.example file.")
        print("   Copy it to create your own .env file:")
        print("   cp .env.example .env")
    
    # Load environment variables from .env file
    load_dotenv()  # Take environment variables from .env
    
    # Check if key variables are available
    api_key = os.environ.get('API_KEY')
    if api_key and api_key != 'your_secret_key_here':
        print("✓ API_KEY found in environment variables")
    else:
        print("⚠️ API_KEY not set or using default value from example")
    
    print("✓ dotenv package loaded successfully!")
    print("✓ Environment variables from .env file are available (if the file exists)")
    
except ImportError:
    print("✗ The python-dotenv package is not installed.")
    print("\nTo install it, run this in your terminal:")
    print("uv pip install python-dotenv")
    print("\nAfter installing, restart the kernel to use it.")

### Accessing Environment Variables

Now let's create a function to access environment variables with proper defaults and type conversion:

**Restaurant equivalent**: This is like having the waiter check if you have any specific dietary preferences or restrictions in their system before taking your order. If they don't find anything, they'll use standard options.

In [ ]:
try:
    import os
    
    # Example using environment variables for API configuration
    def get_api_config():
        """
        Get API configuration from environment variables defined in the .env file
        
        This function uses the variables described in .env.example:
        - API_KEY: Authentication key for the API service
        - API_URL: Base URL for API endpoints
        - API_TIMEOUT: Request timeout in seconds
        - DEBUG: Flag to enable/disable debug mode
        
        Returns:
            dict: Configuration parameters loaded from environment variables with defaults
        """
        config = {
            # Critical authentication parameter
            'api_key': os.environ.get('API_KEY', 'default_key_for_dev'),
            
            # Service endpoint
            'api_url': os.environ.get('API_URL', 'https://api.example.com'),
            
            # Request configuration
            'timeout': int(os.environ.get('API_TIMEOUT', '30')),
            
            # Application behavior
            'debug': os.environ.get('DEBUG', 'False').lower() in ('true', '1', 't'),
            
            # Optional organization ID if needed
            'org_id': os.environ.get('ORGANIZATION_ID', None)
        }
        return config
    
except ImportError:
    print("Error importing required modules")

### Displaying Configuration Securely

**Restaurant equivalent**: This is like having the waiter confirm your order without announcing your credit card details to the entire restaurant. They might say "I've got your payment method on file" rather than reading out your card number.

In [ ]:
try:
    # Show the configuration
    api_config = get_api_config()
    print("API Configuration:")
    for key, value in api_config.items():
        # Mask the API key for security
        if key == 'api_key' and value != 'default_key_for_dev':
            display_value = value[:4] + '*' * (len(value) - 4)
        else:
            display_value = value
        print(f"- {key}: {display_value}")
        
except NameError:
    print("Function get_api_config not available. Make sure to run the previous cell.")

### Environment Variables in Production [OPTIONAL]

**Restaurant equivalent**: This is like how different restaurant locations manage their information systems differently, but the end result is the same - your preferences are stored securely.

> **Note**: This section is optional reading for those interested in professional deployment environments.

In production environments, environment variables are typically set differently:

1. **Cloud Platforms:**
   - AWS: Parameter Store, Secrets Manager
   - Azure: App Configuration, Key Vault
   - Google Cloud: Secret Manager
   - Heroku: Config Vars

2. **Container Environments:**
   - Docker: Environment variables in Dockerfile or docker-compose.yml
   - Kubernetes: ConfigMaps and Secrets

3. **CI/CD Systems:**
   - GitHub Actions: Repository secrets
   - Jenkins: Credentials plugin
   - GitLab CI: CI/CD variables

### Best Practices for Environment Variables [OPTIONAL]

**Restaurant equivalent**: These are like the restaurant's policies for handling customer information:

1. **Never hardcode sensitive information** in your code or commit it to repositories (Don't write customer credit card numbers on napkins)
2. **Use different variables for different environments** (Different procedures for take-out vs. dine-in)
3. **Document required environment variables** in your project's README (Train staff on what customer information they need to collect)
4. **Add `.env` files to your `.gitignore`** to prevent accidental commits (Keep customer information in a locked drawer, not on public display)
5. **Use strong encryption** for sensitive values in production (Use a secure safe for storing customer payment details)
6. **Implement validation** to ensure required variables exist before running critical code (Verify you have all needed information before processing an order)

#### The .env.example Pattern [OPTIONAL]

One best practice we're using in this project is the `.env.example` pattern:

1. **Create a template file** (`.env.example`) with all required variables but dummy values
2. **Commit this template** to version control as documentation
3. **Never commit actual `.env` files** with real values
4. **Instruct new developers** to copy the example and add their own values:
   ```bash
   cp .env.example .env
   # Then edit .env with real values
   ```

This approach ensures that:
- New team members can easily see what environment variables are needed
- No sensitive information is exposed in version control
- Configuration requirements are well-documented

### Conclusion

Environment variables are a powerful tool for API development, allowing you to:
- Keep sensitive credentials secure
- Change configuration without modifying code
- Support different environments (dev, staging, production)
- Follow security best practices

**Restaurant equivalent**: Just as a restaurant needs systems to manage customer preferences, payment details, and location-specific information securely, APIs need environment variables to handle configuration and sensitive data across different contexts.

## 4. Error Handling and Best Practices

When working with APIs, many things can go wrong:
- Network errors
- Authentication failures
- Rate limiting
- Invalid data

**Restaurant equivalent**: Things can go wrong in a restaurant too:
- The kitchen might be closed (server down)
- Your membership card expires (authentication failure)
- The restaurant is too busy to take more orders (rate limiting)
- You ordered something not on the menu (invalid request)

A good waiter knows how to handle these situations gracefully and provide clear explanations to the customer. Similarly, our API client should handle errors gracefully.

Let's build a robust restaurant client with proper problem handling (just like a good restaurant has procedures for handling various situations):

### Restaurant Error Class

First, we'll create a custom exception class to handle restaurant service problems (this is like a restaurant's standardized way of communicating issues to customers):

In [ ]:
class RestaurantError(Exception):
    """Custom exception for restaurant service problems"""
    def __init__(self, message, status_code=None, response=None):
        # message is like the waiter's explanation of what went wrong
        self.message = message
        # status_code is like the specific problem category (kitchen closed, out of ingredient, etc)
        self.status_code = status_code
        # response is the full details of the problem
        self.response = response
        super().__init__(self.message)

### Restaurant Client Class

Below, we're creating a robust restaurant client class that will handle all our interactions with the restaurant's kitchen (API):

**Restaurant equivalent**: This is like setting up your initial preferences when you first sit down at a restaurant - specifying any dietary restrictions, preferred server, or seating area before you start ordering.

Our `RestaurantClient` class is designed to:
1. Maintain a persistent connection to the restaurant kitchen (API)
2. Handle membership verification consistently (authentication)
3. Process orders and responses in a standardized way
4. Handle problems gracefully with detailed explanations
5. Provide convenient methods for different interactions with the waiter

In [ ]:
class RestaurantClient:
    def __init__(self, restaurant_url, membership_card=None):
        # restaurant_url is like the address of the restaurant
        self.base_url = restaurant_url
        # membership_card is like your loyalty card or VIP status
        self.api_key = membership_card
        # session is like your ongoing visit to the restaurant
        self.session = requests.Session()
        
        # Present your membership card when you arrive
        if membership_card:
            self.session.headers.update({
                'Authorization': f'Bearer {membership_card}'
            })
    
    def _make_request(self, method, endpoint, **kwargs):
        """Handle a restaurant interaction with proper error handling"""
        try:
            # Construct the full URL - like determining which part of the restaurant to go to
            url = f"{self.base_url}/{endpoint}".rstrip('/')
            
            # Make the request using our session - like asking the waiter to do something
            response = self.session.request(method, url, **kwargs)
            
            # Check for HTTP errors - like checking if the waiter encountered any problems
            response.raise_for_status()
            
            # Return JSON response if available - like receiving the waiter's answer
            return response.json() if response.content else None
            
        except requests.exceptions.HTTPError as e:
            # Handle different restaurant problems
            if response.status_code == 401:
                raise RestaurantError("Your membership card is invalid or expired", response.status_code, response)
            elif response.status_code == 403:
                raise RestaurantError("This section of the restaurant is reserved for VIP guests", response.status_code, response)
            elif response.status_code == 404:
                raise RestaurantError("The dish you ordered is not on our menu", response.status_code, response)
            elif response.status_code == 429:
                raise RestaurantError("The kitchen is too busy to take more orders right now", response.status_code, response)
            else:
                raise RestaurantError(f"Restaurant problem: {str(e)}", response.status_code, response)
                
        except requests.exceptions.ConnectionError:
            raise RestaurantError("The restaurant appears to be closed")
        except requests.exceptions.Timeout:
            raise RestaurantError("The waiter is taking too long to respond")
        except requests.exceptions.RequestException as e:
            raise RestaurantError(f"Problem communicating with the restaurant: {str(e)}")
        except ValueError:
            raise RestaurantError("The waiter's response was confusing (invalid format)")
            
    # Convenience methods for different restaurant interactions
    def get(self, endpoint, params=None):
        # Like asking the waiter for information
        return self._make_request('GET', endpoint, params=params)

    def post(self, endpoint, data=None, json=None):
        # Like placing a new order
        return self._make_request('POST', endpoint, data=data, json=json)

    def put(self, endpoint, data=None, json=None):
        # Like modifying an existing order
        return self._make_request('PUT', endpoint, data=data, json=json)

    def delete(self, endpoint):
        # Like canceling an order
        return self._make_request('DELETE', endpoint)

### Request Handler with Error Management

**Restaurant equivalent**: This is like the waiter's process for handling your orders. The waiter has a standard procedure: take the order to the kitchen, wait for preparation, check if there are problems, and bring back either your food or an explanation of any issues.

Our request handler has been implemented in the `_make_request` method inside our `RestaurantClient` class above. It:
- Makes the actual HTTP request (waiter takes order to kitchen)
- Handles different types of errors (deals with problems professionally)
- Provides detailed error messages (explains why your dish can't be prepared)
- Formats and returns the response (presents your meal properly)

This centralized method ensures all our API requests follow the same error handling patterns.

### Convenience Methods for HTTP Verbs

Our RestaurantClient class includes convenience methods for common interactions with the restaurant:

**Restaurant equivalent**: These are like the standard interactions you have with a waiter:
- `get()`: Asking for information ("What are today's specials?")
- `post()`: Placing new orders ("I'd like to order the pasta")
- `put()`: Changing existing orders ("Please change my side to salad instead of fries")
- `delete()`: Canceling orders ("I'd like to cancel my dessert")

These methods make our code cleaner by abstracting away the details of HTTP requests behind a simple interface.

### Testing Our API Client

**Restaurant equivalent**: Now that we've set up our relationship with the waiter, let's test it out:
1. We'll visit the restaurant (create a client instance)
2. Order a popular dish (make a successful API call)
3. Try to order something that might not be available (try a potentially failing request)
4. See how the waiter handles the situation (test our error handling)

Let's test our robust API client with a real API:

In [ ]:
# Let's visit our restaurant (JSONPlaceholder API)
try:
    # Create a client instance - like entering the restaurant
    restaurant = RestaurantClient(restaurant_url='https://jsonplaceholder.typicode.com')
    
    # Ask about a menu item - like asking the waiter about a special
    menu_item = restaurant.get(endpoint='posts/1')
    print("Successfully received menu information:")
    print(json.dumps(menu_item, indent=2))
    
    # Try to ask about a dish that doesn't exist on the menu
    menu_item = restaurant.get(endpoint='posts/999999')  # This item number doesn't exist
except RestaurantError as e:
    print(f"Restaurant Error: {e.message}")
    if e.status_code:
        print(f"Problem Code: {e.status_code}")

## 5. Register for SPL

**Restaurant equivalent**: Now that you understand how restaurant service works, it's time to make your own reservation (register for the course)!

In [ ]:
# Place holder for Registration for SPL Logic

## 6. Practice Exercise: Build Your Own API Client

**Restaurant equivalent**: Now it's your turn to be the restaurant manager! Design your own service flow for a new restaurant (API client).

Now it's your turn! Create a restaurant client for the [JSONPlaceholder Restaurant](https://jsonplaceholder.typicode.com/) that can:

1. Get a list of regular customers (API users)
2. View a customer's order history (user's posts)
3. See feedback on particular dishes (comments on a post)
4. Place a new order (create a post)

Requirements:
- Use proper problem handling (similar to the RestaurantClient example)
- Include docstrings and comments
- Format the responses nicely (like a well-presented dish)
- Add some basic input validation (like checking if a dish is available)

Just like a good restaurant needs clear procedures for handling orders, customer information, and feedback, your API client should have well-structured methods for each of these operations.

In [ ]:
class FancyRestaurant:
    def __init__(self):
        self.base_url = 'https://jsonplaceholder.typicode.com'
        # Add your code here - setting up your restaurant visit
    
    def get_regular_customers(self):
        """Get a list of all regular customers (users)"""
        # Add your code here - like getting the guest book
        pass
    
    def get_customer_order_history(self, customer_id):
        """Get order history for a specific customer (user's posts)"""
        # Add your code here - like checking previous orders
        pass
    
    def get_dish_feedback(self, dish_id):
        """Get feedback for a specific dish (comments on a post)"""
        # Add your code here - like reading customer reviews
        pass
    
    def place_new_order(self, customer_id, dish_name, special_instructions):
        """Place a new order (create a post)"""
        # Add your code here - like sending a new order to the kitchen
        pass

# Test your restaurant service implementation here

## Additional Resources

**Restaurant equivalent**: Here's where to find more recipe books and cooking classes to improve your restaurant service!

To learn more about working with APIs in Python:

1. [Requests Library Documentation](https://docs.python-requests.org/)
2. [RESTful API Design Best Practices](https://swagger.io/resources/articles/best-practices-in-api-design/)
3. [Python API Development Fundamentals](https://realpython.com/api-integration-in-python/)
4. [HTTP Status Codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

## Next Steps

**Restaurant equivalent**: Now that you know the basics of restaurant service:
1. Try working at different types of restaurants (build clients for other APIs)
2. Learn different ways to handle VIP customers (experiment with authentication methods)
3. Design your own signature dishes (create your own API)

Now that you understand the basics of working with APIs in Python:
1. Try building clients for other public APIs
2. Experiment with different authentication methods
3. Consider building your own simple API with Flask or FastAPI